In [1]:
!pip install folium

In [3]:
import folium
map_y = folium.Map([37.52860, 126.93431], zoom_start=15)
map_y

In [5]:
import folium
lat, long = 37.52860, 126.93431
map_y = folium.Map([lat, long], zoom_start=15)
folium.Marker([lat, long]).add_to(map_y)
map_y

In [8]:
import folium
lat1, long1 = 37.52860,126.93431 #여의도 한강공원
lat2, long2 = 37.52400, 126.91889 #여의도공원

map_y = folium.Map([lat1, long1],zoom_start=15)
folium.Marker([lat1,long1], tooltip='여의도 한강공원').add_to(map_y)
folium.Marker([lat2,long2], popup='여의도공원').add_to(map_y)
map_y

In [11]:
import folium
lat1, long1 = 37.52860,126.93431 #여의도 한강공원
lat2, long2 = 37.52400, 126.91889 #여의도공원
lat3, long3 = 37.51865, 126.92041 #샛강생태공원

map_y = folium.Map([lat1, long1],zoom_start=15)
folium.Marker([lat1,long1], tooltip='여의도 한강공원', icon=folium.Icon(color='red', icon='automobile', prefix='fa')).add_to(map_y)
folium.Marker([lat2,long2], tooltip='여의도공원', icon=folium.Icon(color='blue', icon='home')).add_to(map_y)
folium.Marker([lat3,long3], tooltip='샛강생태공원', icon=folium.Icon(color='purple', icon='flag')).add_to(map_y)
map_y

In [13]:
import folium

lat = [37.52860,37.52400,37.51865]    # 위도 리스트
long = [126.93431,126.91889,126.92041]    # 경도 리스트
names = ['여의도 한강공원','여의도공원','샛강생태공원']    # 위치 이름 리스트
icons = ['automobile','balance-scale','ban']    # 아이콘 모양 리스트
colors = ['red','blue','purple']    # 아이콘 색상 리스트

map_y = folium.Map([lat[0],long[0]], zoom_start=15)
for i in range(len(lat)):
    folium.Marker(
        [lat[i], long[i]], tooltip=names[i],
        icon=folium.Icon(color=colors[i], icon=icons[i], prefix='fa')
    ).add_to(map_y)
    
map_y

In [15]:
import csv
import operator
import folium

# 인구 데이터 읽기 및 저장
f1 = open('LOCAL_PEOPLE_DONG_202205.csv', encoding='utf8')
data = csv.reader(f1)
next(data)
data = list(data)

# 행정동 코드 데이터 읽기 및 저장
f2 = open('dong_code.csv', encoding='utf8')
code_data = csv.reader(f2)
next(code_data) # 헤더 두 개
next(code_data)
code_data = list(code_data)

# 위치 정보 데이터 읽기 및 저장
f3 = open('dong_location.csv')
location_data = csv.reader(f3)
next(location_data)
location_data = list(location_data)

# 인구 데이터 자료형 변환(문자열->정수)
for row in data:
    for i in range(1,32):
        if i <= 2:
            row[i] = int(row[i])
        else:
            row[i] = float(row[i])
            
# 행정동 코드 데이터 자료형 변환(문자열->정수)
for row in code_data:
    row[0], row[1] = int(row[0]), int(row[1]) # 7자리 코드, 8자리 코드
    
# 위치 정보 데이터 자료형 변환(문자열->정수, 실수)
for row in location_data:
    row[2], row[-2], row[-1] = int(row[2]), float(row[-2]), float(row[-1]) # 행정동코드, 경도, 위도
    
# 행정동별 인구 수 저장 딕셔너리 생성
dong_population = {}

for row in data:
    dong_code, p = row[2],row[3]   # 행정동 코드, 총 생활인구 수
    if dong_code not in dong_population.keys():
        dong_population[dong_code] = p  # dong_code를 키로 하는 항목이 없다면 추가
    else:
        dong_population[dong_code] += p  # dong_code를 키로 하는 항목이 있다면 키 값에 인구수 더함
        
# 총 인구 수 기준 내림차순 정렬
top10 = sorted(dong_population.items(), key = operator.itemgetter(1), reverse = True)[:10]  # 상위 10개 지역 내림차순 정렬

top10_code8 = []
for t in top10:
    top10_code8.append(t[0])  # 딕셔너리(리스트)로 저장된 데이터에서 행정동코드만 저장
    
top10_name = [0 for i in range(10)]
top10_lat = [0 for i in range(10)]  #위도
top10_long = [0 for i in range(10)] #경도

for i in range(len(top10_code8)):
    for row in code_data:            # 8자리 행정동코드에 해당하는 행정동명 찾기
        if row[1] == top10_code8[i]:
            top10_name[i] = row[-1]
            
    for row in location_data:        # 행정동명에 해당하는 위도, 경도 리스트에 저장
        if row[3] == top10_name[i]:
            top10_lat[i] = row[-1]
            top10_long[i] = row[-2]
            
# 행정동명 불일치로 인한 위치정보 누락 시 7자리 코드로 찾기
for i in range(len(top10_lat)):
    if top10_lat[i] == 0 or top10_long[i] == 0:   # 위도, 경도 값이 0이라면
        for row in code_data:    # 7자리 코드를 찾아 변수 dong7에 저장하기
            if top10_code8[i] == row[1]:
                dong7 = row[0]
        
        for row in location_data:  # 7자리 코드가 dong7과 같은 행의 위도, 경도 저장
            if dong7 == row[2]:
                top10_lat[i], top10_long[i] = row[-1], row[-2]
             
# 순위 표시
for i in range(len(top10_name)):
    top10_name[i] = str(i+1) + '. ' + top10_name[i]
            
# 지도 작성
map_top10 = folium.Map([top10_lat[0], top10_long[0]], zoom_start=12)
for i in range(10):
    folium.Marker(
        [top10_lat[i], top10_long[i]],tooltip = top10_name[i],
        icon = folium.Icon(color='red', icon='star')
    ).add_to(map_top10)
map_top10

In [22]:
import csv
import operator
import folium

# 인구 데이터 읽기 및 저장
f1 = open('LOCAL_PEOPLE_DONG_202205.csv', encoding='utf8')
data = csv.reader(f1)
next(data)
data = list(data)

# 행정동 코드 데이터 읽기 및 저장
f2 = open('dong_code.csv', encoding='utf8')
code_data = csv.reader(f2)
next(code_data) # 헤더 두 개
next(code_data)
code_data = list(code_data)

# 위치 정보 데이터 읽기 및 저장
f3 = open('dong_location.csv')
location_data = csv.reader(f3)
next(location_data)
location_data = list(location_data)

# 인구 데이터 자료형 변환(문자열->정수)
for row in data:
    for i in range(1,32):
        if i <= 2:
            row[i] = int(row[i])
        else:
            row[i] = float(row[i])
            
# 행정동 코드 데이터 자료형 변환(문자열->정수)
for row in code_data:
    row[0], row[1] = int(row[0]), int(row[1]) # 7자리 코드, 8자리 코드
    
# 위치 정보 데이터 자료형 변환(문자열->정수, 실수)
for row in location_data:
    row[2], row[-2], row[-1] = int(row[2]), float(row[-2]), float(row[-1]) # 행정동코드, 경도, 위도
    
# 행정동별 남성 인구 수 저장 딕셔너리 생성
dong_population = {}

for row in data:
    dong_code, p = row[2],sum(row[4:18])   # 행정동 코드, 남성인구수
    if dong_code not in dong_population.keys():
        dong_population[dong_code] = p  # dong_code를 키로 하는 항목이 없다면 추가
    else:
        dong_population[dong_code] += p  # dong_code를 키로 하는 항목이 있다면 키 값에 인구수 더함
        
# 총 인구 수 기준 내림차순 정렬
top10 = sorted(dong_population.items(), key = operator.itemgetter(1), reverse = True)[:10]  # 상위 10개 지역 내림차순 정렬

top10_code8 = []
for t in top10:
    top10_code8.append(t[0])  # 딕셔너리(리스트)로 저장된 데이터에서 행정동코드만 저장
    
top10_name = [0 for i in range(10)]
top10_lat = [0 for i in range(10)]  #위도
top10_long = [0 for i in range(10)] #경도

for i in range(len(top10_code8)):
    for row in code_data:            # 8자리 행정동코드에 해당하는 행정동명 찾기
        if row[1] == top10_code8[i]:
            top10_name[i] = row[-1]
            
    for row in location_data:        # 행정동명에 해당하는 위도, 경도 리스트에 저장
        if row[3] == top10_name[i]:
            top10_lat[i] = row[-1]
            top10_long[i] = row[-2]
            
# 행정동명 불일치로 인한 위치정보 누락 시 7자리 코드로 찾기
for i in range(len(top10_lat)):
    if top10_lat[i] == 0 or top10_long[i] == 0:   # 위도, 경도 값이 0이라면
        for row in code_data:    # 7자리 코드를 찾아 변수 dong7에 저장하기
            if top10_code8[i] == row[1]:
                dong7 = row[0]
        
        for row in location_data:  # 7자리 코드가 dong7과 같은 행의 위도, 경도 저장
            if dong7 == row[2]:
                top10_lat[i], top10_long[i] = row[-1], row[-2]
                
# 순위 표시   
for i in range(len(top10_name)):
    top10_name[i] = str(i+1) + '. ' + top10_name[i]
            
# 지도 작성
map_top10 = folium.Map([top10_lat[0], top10_long[0]], zoom_start=12)
for i in range(10):
    if i < 5:
        folium.Marker(
            [top10_lat[i], top10_long[i]],tooltip = top10_name[i],
            icon = folium.Icon(color='darkblue', icon='male', prefix='fa')
        ).add_to(map_top10)
    else:
        folium.Marker(
            [top10_lat[i], top10_long[i]],tooltip = top10_name[i],
            icon = folium.Icon(color='lightblue', icon='male', prefix='fa')
        ).add_to(map_top10)
map_top10

In [20]:
# import csv
import operator
import folium

# 인구 데이터 읽기 및 저장
f1 = open('LOCAL_PEOPLE_DONG_202205.csv', encoding='utf8')
data = csv.reader(f1)
next(data)
data = list(data)

# 행정동 코드 데이터 읽기 및 저장
f2 = open('dong_code.csv', encoding='utf8')
code_data = csv.reader(f2)
next(code_data) # 헤더 두 개
next(code_data)
code_data = list(code_data)

# 위치 정보 데이터 읽기 및 저장
f3 = open('dong_location.csv')
location_data = csv.reader(f3)
next(location_data)
location_data = list(location_data)

# 인구 데이터 자료형 변환(문자열->정수)
for row in data:
    for i in range(1,32):
        if i <= 2:
            row[i] = int(row[i])
        else:
            row[i] = float(row[i])
            
# 행정동 코드 데이터 자료형 변환(문자열->정수)
for row in code_data:
    row[0], row[1] = int(row[0]), int(row[1]) # 7자리 코드, 8자리 코드
    
# 위치 정보 데이터 자료형 변환(문자열->정수, 실수)
for row in location_data:
    row[2], row[-2], row[-1] = int(row[2]), float(row[-2]), float(row[-1]) # 행정동코드, 경도, 위도
    
# 행정동별 여성 인구 수 저장 딕셔너리 생성
dong_population = {}

for row in data:
    dong_code, p = row[2],sum(row[18:32])   # 행정동 코드, 여성인구수
    if dong_code not in dong_population.keys():
        dong_population[dong_code] = p  # dong_code를 키로 하는 항목이 없다면 추가
    else:
        dong_population[dong_code] += p  # dong_code를 키로 하는 항목이 있다면 키 값에 인구수 더함
        
# 총 인구 수 기준 내림차순 정렬
top10 = sorted(dong_population.items(), key = operator.itemgetter(1), reverse = True)[:10]  # 상위 10개 지역 내림차순 정렬

top10_code8 = []
for t in top10:
    top10_code8.append(t[0])  # 딕셔너리(리스트)로 저장된 데이터에서 행정동코드만 저장
    
top10_name = [0 for i in range(10)]
top10_lat = [0 for i in range(10)]  #위도
top10_long = [0 for i in range(10)] #경도

for i in range(len(top10_code8)):
    for row in code_data:            # 8자리 행정동코드에 해당하는 행정동명 찾기
        if row[1] == top10_code8[i]:
            top10_name[i] = row[-1]
            
    for row in location_data:        # 행정동명에 해당하는 위도, 경도 리스트에 저장
        if row[3] == top10_name[i]:
            top10_lat[i] = row[-1]
            top10_long[i] = row[-2]
            
# 행정동명 불일치로 인한 위치정보 누락 시 7자리 코드로 찾기
for i in range(len(top10_lat)):
    if top10_lat[i] == 0 or top10_long[i] == 0:   # 위도, 경도 값이 0이라면
        for row in code_data:    # 7자리 코드를 찾아 변수 dong7에 저장하기
            if top10_code8[i] == row[1]:
                dong7 = row[0]
        
        for row in location_data:  # 7자리 코드가 dong7과 같은 행의 위도, 경도 저장
            if dong7 == row[2]:
                top10_lat[i], top10_long[i] = row[-1], row[-2]
                
# 순위 표시   
for i in range(len(top10_name)):
    top10_name[i] = str(i+1) + '. ' + top10_name[i]
            
# 지도 작성
map_top10 = folium.Map([top10_lat[0], top10_long[0]], zoom_start=12)
for i in range(10):
    if i < 5:
        folium.Marker(
            [top10_lat[i], top10_long[i]],tooltip = top10_name[i],
            icon = folium.Icon(color='red', icon='female', prefix='fa')
        ).add_to(map_top10)
    else:
        folium.Marker(
            [top10_lat[i], top10_long[i]],tooltip = top10_name[i],
            icon = folium.Icon(color='pink', icon='female', prefix='fa')
        ).add_to(map_top10)
map_top10

# Self Check

In [24]:
#실습1번
import folium

lat, long = 35.15870, 129.16047
map_y = folium.Map([lat,long], zoom_start=11)
folium.Marker([lat,long], tooltip='해운대 해수욕장', icon=folium.Icon(color='blue', icon='hand-o-down', prefix='fa')).add_to(map_y)
map_y

In [26]:
#실습2번
import folium

lat1, long1 = 35.15870, 129.16047   # 해운대 해수욕장
lat2, long2 = 35.23360, 129.08309   # 부산대학교
lat3, long3 = 35.17357, 128.94632   # 김해국제공항

map_y = folium.Map([lat1,long1], zoom_start=11)
folium.Marker([lat1,long1], tooltip='해운대 해수욕장', icon=folium.Icon(color='blue', icon='hand-o-down', prefix='fa')).add_to(map_y)
folium.Marker([lat2,long2], tooltip='부산대학교', icon=folium.Icon(color='blue', icon='university', prefix='fa')).add_to(map_y)
folium.Marker([lat3,long3], tooltip='김해국제공항', icon=folium.Icon(color='blue', icon='plane', prefix='fa')).add_to(map_y)
map_y